In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
mat_df = pd.read_csv('../datasets/drug_drug_matrix.csv')
druglist = pd.read_csv('../datasets/description about features/drug_list.txt', sep=' ')

In [3]:
druglist = druglist['DBid'].values

In [4]:
CIDs = mat_df.columns.values[1:]

In [5]:
CIDs = [int(i.replace('CID','') )for i in CIDs]

In [6]:
mat = mat_df.values[:, 1:]

In [7]:
edges = []
for i in tqdm(range(len(CIDs))):
    for j in range(i, len(CIDs)):
        if i != j :
            edges.append(np.array([i, j, mat[i][j]]))
edges = np.array(edges)        

100%|██████████| 548/548 [00:00<00:00, 2666.17it/s]


In [8]:
druglist[0]

'DB00945'

In [35]:
db = pd.read_csv('../datasets/dbsmiles.csv')

In [38]:
def returnsmiles(dbid):
    a = db.loc[db['drugbank_id'] == dbid]
    print(list(a))
    return list(a['smiles'])[0]

In [ ]:

a = db['drugbank_id'].values
final_edges = []
for i in tqdm(edges):
    d1 = i[0]
    d2 = i[1]
    if druglist[d1] in a:
        if druglist[d2] in a:
            final_edges.append([returnsmiles(druglist[d1]), 
                                returnsmiles(druglist[d2]),
                                i[2]])
        
    

In [ ]:
final_edges

In [ ]:
final_edges = np.array(final_edges)

In [ ]:
final_edges.dump('../datasets/final_edges.dump')

In [10]:
final_edges = np.load('../datasets/final_edges.dump', allow_pickle=True)

In [11]:
from rdkit.Chem import AllChem

In [18]:
from rdkit import Chem
import rdkit

In [47]:
m = Chem.MolFromSmiles('NCC1(CC(O)=O)CCCCC1')

In [48]:
fp1 = rdkit.Chem.RDKFingerprint(m, maxPath=7,fpSize=512)

In [42]:
final_edges

array([['CC(=O)OC1=CC=CC=C1C(O)=O', 'NCC1(CC(O)=O)CCCCC1', '1'],
       ['CC(=O)OC1=CC=CC=C1C(O)=O',
        'CNS(=O)(=O)CCC1=CC2=C(NC=C2C2CCN(C)CC2)C=C1', '0'],
       ['CC(=O)OC1=CC=CC=C1C(O)=O',
        'COC1=C(OC)C(CS(=O)C2=NC3=C(N2)C=C(OC(F)F)C=C3)=NC=C1', '1'],
       ...,
       ['CCN[C@H]1CN(CCCOC)S(=O)(=O)C2=C1C=C(S2)S(N)(=O)=O',
        '[H][C@@]12CO[C@@]3(COS(N)(=O)=O)OC(C)(C)O[C@@]3([H])[C@]1([H])OC(C)(C)O2',
        '0'],
       ['CCN[C@H]1CN(CCCOC)S(=O)(=O)C2=C1C=C(S2)S(N)(=O)=O',
        'NCCCC(O)=O', '0'],
       ['[H][C@@]12CO[C@@]3(COS(N)(=O)=O)OC(C)(C)O[C@@]3([H])[C@]1([H])OC(C)(C)O2',
        'NCCCC(O)=O', '0']], dtype='<U285')

In [59]:
data = []
for edge in tqdm(final_edges):
    mol1 = Chem.MolFromSmiles(edge[0])
    mol2 = Chem.MolFromSmiles(edge[1])
    fp1 = np.array(rdkit.Chem.RDKFingerprint(mol1, maxPath=7, fpSize=512), dtype=object)
    fp2 = np.array(rdkit.Chem.RDKFingerprint(mol2, maxPath=7, fpSize=512), dtype=object)
    data.append(np.array([fp1,fp2,edge[2]], dtype=object))
data = np.array(data)

100%|██████████| 87153/87153 [06:55<00:00, 209.91it/s]


In [62]:
data.dump('fingerprint_data.dump')

In [64]:
data.shape

(87153, 3)

In [65]:
X = []
y = []
for i in tqdm(data):
    X.append(np.concatenate((i[0],i[1]), axis = None))
    y.append(i[2])

100%|██████████| 87153/87153 [00:00<00:00, 137558.86it/s]


In [68]:
X = np.array(X)
y = np.array(y)

In [69]:
X.shape

(87153, 1024)

In [70]:
y.shape

(87153,)

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [72]:
from sklearn.ensemble import RandomForestClassifier
reg_rf = RandomForestClassifier()
reg_rf.fit(X_train, y_train)
y_pred = reg_rf.predict(X_test)

In [74]:
from sklearn import metrics

In [75]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86     11650
           1       0.76      0.62      0.68      5781

    accuracy                           0.81     17431
   macro avg       0.79      0.76      0.77     17431
weighted avg       0.80      0.81      0.80     17431



In [ ]:
print("roc_auc_score: ", metrics.roc_auc_score(y_test, y_pred))
print("f1 score: ", metrics.f1_score(y_test, y_pred))

In [ ]:
from sklearn.svm import SVC
reg_svc = SVC()
reg_svc.fit(X_train, y_train)
y_pred = reg_svc.predict(X_test)
print(metrics.classification_report(y_test, y_pred))


In [ ]:
print("roc_auc_score: ", metrics.roc_auc_score(y_test, y_pred))
print("f1 score: ", metrics.f1_score(y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
reg_knn = KNeighborsClassifier()
reg_knn.fit(X_train, y_train)
y_pred = reg_knn.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))


In [ ]:
print("roc_auc_score: ", metrics.roc_auc_score(y_test, y_pred))
print("f1 score: ", metrics.f1_score(y_test, y_pred))